# Get started with Keras callbacks
1. **ModelCheckpoint**
2. Tensorbaord
3. LearningRateScheduler
4. EarlyStopping

## Librairies

In [1]:
import os

import matplotlib.pyplot as plt
import tensorflow as tf
tf.__version__

'2.4.1'

## Paths & Config

In [2]:
DATA_DIR = "data/flower_photos/"
CHECKPOINT = "/tmp/Get-started-with-Keras-callbacks-classifier-weights.hdf5"

VALIDATION_SPLIT = 0.2

WIDTH = 150
HEIGHT = 150
NUM_CHANNELS = 3
INPUT_SHAPE = (WIDTH, HEIGHT, NUM_CHANNELS)

LEARNING_RATE = 1e-4

EPOCHS = 10
BATCH_SIZE = 32

## Data

In [3]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=VALIDATION_SPLIT
)
train_set = generator.flow_from_directory(
    DATA_DIR,
    target_size=(WIDTH, HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)
val_set = generator.flow_from_directory(
    DATA_DIR,
    target_size=(WIDTH, HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)
num_classes = len(train_set.class_indices)
print("Total classes:", num_classes)

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.
Total classes: 5


## Model Transfer Learning

In [4]:
def build_model(input_shape, num_classes):
    base_model = tf.keras.applications.vgg16.VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    # Freeze base model
    base_model.trainable = False

    # Build model
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(units=num_classes, activation='softmax')
    ])

    return model

model = build_model(INPUT_SHAPE, num_classes)
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              8389632   
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 5125      
Total params: 23,109,445
Trainable params: 8,394,757
Non-trainable params: 14,714,688
_________________________________________________________________
None


## Restore checkpoints

In [5]:
if os.path.exists(CHECKPOINT):
    try:
        _ = model.load_weights(CHECKPOINT, by_name=False)
    except:
        pass

## Loss & Optimizer

In [6]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.RMSprop(lr=LEARNING_RATE),
              metrics=["acc"])

## Callbacks

In [7]:
callback_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=CHECKPOINT,
    verbose=0,
    save_weights_only=True
)

callbacks = [callback_checkpoint]

## Training

In [ ]:
history = model.fit(
    train_set,
    epochs=EPOCHS,
    validation_data=val_set,
    callbacks=callbacks
)

Epoch 1/10
92/92 [==============================] - 3237s 35s/step - loss: 1.1457 - acc: 0.5750 - val_loss: 0.8458 - val_acc: 0.6881
Epoch 2/10
81/92 [=========================>....] - ETA: 7:18 - loss: 0.5112 - acc: 0.8290

In [ ]:
acc = history.history['acc']
loss = history.history['loss']
val_acc = history.history['val_acc']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(8, 4))
plt.plot(epochs, acc, 'bo', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='best')
plt.show()

plt.figure(figsize=(8, 4))
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')
plt.legend(loc='best')
plt.show()